In [3]:
from langchain_google_genai import GoogleGenerativeAIEmbeddings
import os
from langchain_community.vectorstores import Neo4jVector

os.environ["GOOGLE_API_KEY"] = os.environ["GEMINI_API"]

embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")

In [4]:
# Create the vectorstore for our existing graph
fair_use_graph = Neo4jVector.from_existing_graph(
    embedding = embeddings,
    url = "bolt://localhost:7687",
    username = "neo4j",
    password = "fairusecases",
    node_label = ["Opinion", "Facts", "Market", "Conclusion", "Amount", "Nature", "Purpose"],
    text_node_properties = ["Document"],
    embedding_node_property="embedding",
)

ValueError: Cypher Statement is not valid
{code: Neo.ClientError.Statement.SyntaxError} {message: Type mismatch for parameter 'node_label': expected String but was List<String> (line 1, column 50 (offset: 49))
"CALL db.index.vector.createNodeIndex($index_name,$node_label,$embedding_node_property,toInteger($embedding_dimension),$similarity_metric )"
                                                  ^}